<a href="https://colab.research.google.com/github/nupursjsu/Deep-Learning/blob/master/Ungraded_Assignment5/Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing necessary libraries

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

## Loading and Preprocessing MNIST dataset

In [0]:
df_mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = df_mnist.load_data()

#Scaling the images
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
#Adding a channels dimension
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

In [0]:
#Creating batch and shuffling the dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

## Building the model

In [0]:
#Building the Keras model
class My_Model(Model):
  def __init__(self):
    super(My_Model, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.dense1 = Dense(128, activation='relu')
    self.dense2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.dense1(x)
    return self.dense2(x)

In [0]:
#Creating an instance of the model
model = My_Model()

In [0]:
#Defining optimizer and loss function for training
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [0]:
#Defining metrics to measure the loss and the accuracy of the model
trainData_loss = tf.keras.metrics.Mean(name='trainData_loss')
trainData_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='trainData_accuracy')

testData_loss = tf.keras.metrics.Mean(name='testData_loss')
testData_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='testData_accuracy')

In [0]:
#Training the model using tf.GradientTape
@tf.function
def train_model(imgs, labels):
  with tf.GradientTape() as tape:
    preds = model(imgs, training=True)
    loss = loss_obj(labels, preds)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  trainData_loss(loss)
  trainData_accuracy(labels, preds)

In [0]:
#Testing the model
@tf.function
def test_model(imgs, labels):
  preds = model(imgs, training=False)
  test_loss = loss_obj(labels, preds)

  testData_loss(test_loss)
  testData_accuracy(labels, preds)

In [48]:
#Running the model with 5 epochs
EPOCHS = 5

for i in range(EPOCHS):
  #Reseting the metrics at the start of the next epoch
  trainData_loss.reset_states()
  trainData_accuracy.reset_states()
  testData_loss.reset_states()
  testData_accuracy.reset_states()

  for imgs, labels in train_ds:
    train_model(imgs, labels)

  for test_imgs, test_labels in test_ds:
    test_model(test_imgs, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(i + 1,
                        trainData_loss.result(),
                        trainData_accuracy.result() * 100,
                        testData_loss.result(),
                        testData_accuracy.result() * 100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.14168325066566467, Accuracy: 95.71833801269531, Test Loss: 0.05956956744194031, Test Accuracy: 98.02999877929688
Epoch 2, Loss: 0.04415746405720711, Accuracy: 98.62166595458984, Test Loss: 0.051343198865652084, Test Accuracy: 98.32999420166016
Epoch 3, Loss: 0.022704968228936195, Accuracy: 99.22999572753906, Test Loss: 0.06628067791461945, Test Accuracy: 98.02999877929688
Epoch 4, Loss: 0.014348485507071018, Accuracy: 99.54166412353516, Test Loss: 0.06172284856438637, Test Accuracy: 98.

## Conclusion

Our image classifier model is giving an accuracy of approx `98%` on MNIST dataset.